# Imports

In [4]:
import keras
import numpy as np
import gymnasium as gym
import tensorflow as tf
import tqdm.notebook as tqdm
import matplotlib.pyplot as plt
from typing import Optional, Union, Tuple
import ale_py
import numpy as np 
from PIL import Image
import gymnasium.utils.save_video

In [5]:
gpus = tf.config.experimental.list_physical_devices('GPU')
if gpus:
    try:
        for gpu in gpus:
              tf.config.experimental.set_memory_growth(gpu, True)
    except RuntimeError as e:
        print(e)

In [6]:
def get_name(prefix: Optional[str] = None, suffix: Optional[str] = None, separator: str = '_') -> Optional[str]:
    return prefix and prefix + separator + suffix or suffix or None

# Environment

Create the environment. You can use any ATARI environment from [here](https://gymnasium.farama.org/environments/atari/), but prefer to use environments with discrete action space with fewer actions.

In [7]:
batch_size = 64

In [8]:
gym.register_envs(ale_py)

In [9]:
train_env = gym.make_vec("ALE/Boxing-v5", render_mode='rgb_array', num_envs=batch_size)
valid_env = gym.make_vec("ALE/Boxing-v5", render_mode='rgb_array', num_envs=1)

A.L.E: Arcade Learning Environment (version 0.10.2+c9d4b19)
[Powered by Stella]


# Replay Buffer

Create a replay buffer to hold game history

In [10]:
class ReplayBuffer:

    def __init__(self, max_size: int, observation_space: gym.spaces.Space, action_space: gym.spaces.Space, seed: Optional[int] = None):
        """Stores the replay history with a maximum of `max_size` entries, removing old entries as needed.

        Parameters:
            max_size: maximal number of entries to keep
            observation_space: specification of the observation space
            action_space: specification of the action space
            seed: seed to initialize the internal random number generator for reproducibility"""

        self.current_observations = np.zeros((max_size, *observation_space.shape), dtype=observation_space.dtype)
        self.next_observations = np.zeros((max_size, *observation_space.shape), dtype=observation_space.dtype)
        self.actions = np.zeros((max_size, *action_space.shape), dtype=action_space.dtype)
        self.rewards = np.zeros((max_size,), dtype=np.float32)
        self.dones = np.zeros((max_size,), dtype=np.float32)
        
        self.max_size = max_size
        self.rng = np.random.default_rng(seed=seed)
        self.buffer_pointer = 0
        self.current_size = 0
        
    def add(self, current_observations: np.ndarray, actions: np.ndarray, rewards: np.ndarray, next_observations: np.ndarray, dones: np.ndarray) -> None:
        """Add a new entry to the buffer.

        Parameters:
            current_observations: environment state observed at the current step
            actions: action taken by the model
            rewards: reward received after taking the action
            next_observations: environment state obversed after taking the action
            dones: whether the episode has ended or not"""

        batch_size = current_observations.shape[0]
        idxs = (np.arange(batch_size) + self.buffer_pointer) % self.max_size

        self.current_observations[idxs] = current_observations
        self.actions[idxs] = actions
        self.rewards[idxs] = rewards
        self.next_observations[idxs] = next_observations
        self.dones[idxs] = dones

        self.buffer_pointer = (idxs[-1] + 1) % self.max_size
        self.current_size = min(self.max_size, self.current_size + batch_size)
    
    def sample(self, n_samples: int, replace: bool = True) -> tuple[np.ndarray, np.ndarray, np.ndarray, np.ndarray, np.ndarray]:
        """Randomly samples `n_samples` from the buffer.

        Parameters:
            n_samples: number of samples to select
            replace: sample with or without replacement

        Returns:
            current observations, actions, rewards, next observations, dones"""

        return self[self.rng.choice(self.current_size, size=n_samples, replace=replace)]

    def clear(self) -> None:
        """Clears the buffer"""

        self.buffer_pointer = 0
        self.current_size = 0

    def __getitem__(self, index: Union[int , np.ndarray]) -> tuple[np.ndarray, np.ndarray, np.ndarray, np.ndarray, np.ndarray]:
        """Gets a sample at `index`

        Parameters:
            index: index of the sample to get

        Returns:
            current observations, actions, rewards, next observations, dones"""

        return (
            self.current_observations[index],
            self.actions[index],
            self.rewards[index],
            self.next_observations[index],
            self.dones[index]
        )
        
    def __len__(self) -> int:
        """Returns the number of entries in the buffer"""

        return self.current_size

# Model

Implement your model. Most if not all ATARI environments have an image observation

In [11]:
 def get_model(
    observation_space: gym.spaces.Space,
    action_space: gym.spaces.Space,
    features: int,
    blocks: int, 
    activation: str | keras.layers.Activation | None = 'relu',
    dropout_rate: float = 0.,
    multiply_freq: int = 1,
    name: str | None = None
 ) -> keras.Model:
    inputs = keras.layers.Input(observation_space.shape, name=get_name(name, "Input"), dtype=observation_space.dtype)

    x = keras.layers.Rescaling(1.0 / 255, name=get_name(name, "Rescaling"))(inputs)

    for idx in range(blocks):
        features = (idx + 1) * features

        x = keras.layers.Conv2D(features, 3, padding="same", activation=activation, name=get_name(name, f"Conv2D_{idx}"))(x)
        x = keras.layers.MaxPooling2D((2, 2), name=get_name(name, f"MaxPooling2D_{idx}"))(x)
        
    x = keras.layers.BatchNormalization(name=get_name(name, f"BatchNormalization_{idx}"))(x)
    x = keras.layers.GlobalAveragePooling2D(name=get_name(name, "GlobalAveragePooling2D"))(x)
        
    outputs = keras.layers.Dense(action_space.n, name=get_name(name, "Output"))(x)
    
    return keras.Model(inputs, outputs, name=name)

# Sampler

Implement the sampler

In [12]:
class Sampler:
    
    def __init__(self, epsilon: float, seed: int | None = None, greedy: bool = False):
        """Selects a random action with probability `epsilon` otherwise if `greedy` selects the most probable action given by the model,
        if not `greedy` samples the action from the distribution given by the model.

        Parameters:
            epsilon: the probability to select a random action
            seed: seed to initialize the internal random number generator for reproducibility
            greedy: select the most probable action"""
        self.epsilon = epsilon
        self.greedy = greedy
        self.rng = np.random.default_rng(seed=seed)

    def _sample(self, probabilities: np.ndarray) -> np.ndarray:
        cumsum = np.cumsum(probabilities, axis=-1)
        probs = self.rng.uniform(size=(cumsum.shape[0], 1))
        msk = cumsum > probs
        return np.argmax(msk, axis=-1)

    def __call__(self, probabilities: np.ndarray) -> np.ndarray:
        """Select an action given the `probabilities`

        Parameters:
            probabilities: batch of probabilities for each action

        Returns:
            batch of indices of the selected actions"""
        batch, *_ = probabilities.shape
        probs = self.rng.uniform(size=batch)

        return np.where(
            probs < self.epsilon,
            self.rng.choice(probabilities.shape[1], size=batch),
            np.argmax(probabilities, axis=-1) if self.greedy else self._sample(probabilities),
        )

# Play the game

Implement interacting with the environment and storing entries to the replay buffer

In [13]:
def play_game(
    model: keras.Model,
    buffer: ReplayBuffer,
    env: gym.Env,
    steps: int,
    sampler: Sampler,
    observations: np.ndarray | None = None,
    one_episode: bool = False
) -> np.ndarray:
    """Plays the environment `env` using model `model` for a total of `steps` steps.

    Arguments:
        model: model to use
        buffer: buffer to store results to
        env: environment to use
        steps: total number of steps to record
        sampler: sampler to use
        observations: observation to start from
        one_episode: exist as soon as one of the environments finishes

    Returns:
        the last observations
    """
    if observations is None:
        observations, _ = env.reset()

    for _ in range(steps // env.num_envs):
        res = model(observations, training=False).numpy()
        actions = sampler(res)
        new_observations, rewards, terminateds, truncated, _ = env.step(actions) # new_observations, rewards, terminateds, truncated, info

        dones = terminateds | truncated

        buffer.add(
            current_observations=observations,
            actions=actions,
            rewards=rewards,
            next_observations=new_observations,
            dones=dones,
        )
        
        if one_episode and np.any(dones):
            observations = None
            break
        
        observations = new_observations
    
    return observations

# Loss

Implement double q learning loss. Don't forget to stop the gradient for q_ref

In [14]:
class QDataset(keras.utils.PyDataset):

    def __init__(
        self,
        steps_per_epoch: int,
        batch_size: int,
        buffer: ReplayBuffer,
        action_space: gym.spaces.Space,
    ):
        super().__init__()
        self.buffer = buffer
        self.batch_size = batch_size
        self.steps_per_epoch = steps_per_epoch

        self._eye = np.eye(action_space.n)
        self._answer = np.zeros(batch_size, dtype=np.float32)

    def __len__(self):
        return self.steps_per_epoch

    def __getitem__(self, idx: int) -> np.ndarray:
        a, b, c, d, e = self.buffer.sample(self.batch_size)
        b = self._eye[b]

        return (a, b, c, d, e), self._answer

In [15]:
class QLoss(keras.layers.Layer):

    def __init__(self, gamma: float, **kwargs):
        super().__init__(**kwargs)
        
        self.gamma = gamma

    def call(
        self,
        q_current: keras.KerasTensor,
        q_next: keras.KerasTensor,
        rewards: keras.KerasTensor,
        actions: keras.KerasTensor,
        dones: keras.KerasTensor
    ) -> keras.KerasTensor:
        q_ref = rewards + self.gamma * (1 - dones) * keras.ops.max(q_next, axis=-1)

        return keras.ops.square(keras.ops.sum(q_current * actions, axis=-1) - q_ref)

    def get_config(self) -> dict:
        config = super().get_config()
        config['gamma'] = self.gamma
        return config

In [16]:
def get_combined_model(
    observation_space: gym.spaces.Space,
    action_space: gym.spaces.Space,    
    features: int,
    blocks: int, 
    activation: str | keras.layers.Activation | None = 'relu',
    dropout_rate: float = 0.,
    multiply_freq: int = 1,
    name: str | None = None,
    gamma: float = 0.99
) -> keras.Model:
    """Creates a combined model for Q-learning training

    Arguments:
        input_features: model input vector size
        features: initial model embedding size
        out_features: model output vector size
        block: number of perceptron layers
        activation: intermediate model activation
        dropout_rate: nuff said
        multifly_freq: doubles embedding size every `multiply_freq` blocks
        name: model name,
        gamma: rewards discount
    Returns:
        A q-model, a target q-model, a combined model
    """
    model = get_model(
        observation_space=observation_space,
        action_space=action_space,
        features=features,
        blocks=blocks,
        activation=activation,
        dropout_rate=dropout_rate,
        multiply_freq=multiply_freq,
        name=get_name(name, "model"),
    )
    
    target_model = get_model(
        observation_space=observation_space,
        action_space=action_space,
        features=features,
        blocks=blocks,
        activation=activation,
        multiply_freq=multiply_freq,
        name=get_name(name, "target_model"),
    )

    target_model.trainable = False
    target_model.set_weights(model.get_weights())

    current_observation = keras.layers.Input(observation_space.shape, dtype=observation_space.dtype, name=get_name(name, "curr_observ"))
    next_observation = keras.layers.Input(observation_space.shape, dtype=observation_space.dtype, name=get_name(name, "next_observ"))
    current_action = keras.layers.Input((action_space.n,), dtype="float32", name=get_name(name, "curr_action"))
    rewards = keras.layers.Input((), dtype="float32", name=get_name(name, "rewards"))
    dones = keras.layers.Input((), dtype="float32", name=get_name(name, "dones"))

    model_res = model(current_observation)
    target_model_res = target_model(next_observation)

    loss = QLoss(gamma=gamma, name=get_name(name, "Q_loss"))(model_res, target_model_res, rewards, current_action, dones)

    return (
        model, 
        target_model, 
        keras.Model(
            inputs=[
                current_observation,
                current_action,
                rewards,
                next_observation,
                dones,
            ],
            outputs=loss,
            name=name
        ),
    )

In [17]:
def play_game(
    model: keras.Model,
    buffer: ReplayBuffer,
    env: gym.Env,
    steps: int,
    sampler: Sampler,
    observations: np.ndarray | None = None,
    one_episode: bool = False
) -> np.ndarray:
    """Plays the environment `env` using model `model` for a total of `steps` steps.

    Arguments:
        model: model to use
        buffer: buffer to store results to
        env: environment to use
        steps: total number of steps to record
        sampler: sampler to use
        observations: observation to start from
        one_episode: exist as soon as one of the environments finishes

    Returns:
        the last observations
    """
    if observations is None:
        observations, _ = env.reset()

    for _ in range(steps // env.num_envs):
        res = model(observations, training=False).numpy()
        actions = sampler(res)
        new_observations, rewards, terminateds, truncated, _ = env.step(actions) # new_observations, rewards, terminateds, truncated, info

        dones = terminateds | truncated

        buffer.add(
            current_observations=observations,
            actions=actions,
            rewards=rewards,
            next_observations=new_observations,
            dones=dones,
        )
        
        if one_episode and np.any(dones):
            observations = None
            break
        
        observations = new_observations
    
    return observations

In [18]:
class ModelUpdateCallback(keras.callbacks.Callback):

    def __init__(self, frequency: int, **kwargs):
        super().__init__(**kwargs)
        self.frequency = frequency
        
    def on_train_batch_end(self, batch: int, logs=None):
        if (batch + 1) % self.frequency == 0:
            self.model.get_layer(
                f'{self.model.name}_target_model'
            ).set_weights(
                self.model.get_layer(
                    f'{self.model.name}_model'
                ).get_weights()
            )

In [19]:
class ModelEvalCallback(keras.callbacks.Callback):

    def __init__(self, env: gym.Env, max_steps: int, **kwargs):
        super().__init__(**kwargs)
        self.env = env
        self.max_steps = max_steps

        self.buffer = ReplayBuffer(
            max_size=max_steps,
            observation_space=env.single_observation_space,
            action_space=env.single_observation_space,
        )

        self.sampler = Sampler(epsilon=0., greedy=True)

    def on_epoch_end(self, epoch: int, logs=None):
        if logs is None:
            return

        model = self.model.get_layer(
            f'{self.model.name}_model'
        )

        self.buffer.clear()

        play_game(
            model=model,
            buffer=self.buffer,
            env=self.env,
            steps=self.max_steps,
            sampler=self.sampler,
            one_episode=True,
        )

        rewards = self.buffer.rewards[:len(self.buffer)].reshape((-1, self.env.num_envs))
        logs['score'] = np.mean(np.sum(rewards, axis=0))

In [20]:
class SometimesPlayCallback(keras.callbacks.Callback):
    def __init__(
        self,
        env: gym.Env,
        buffer: ReplayBuffer,
        sampler: Sampler,
        frequency: int,
        steps_per_play: int,
        epsilon_decay: keras.optimizers.schedules.PolynomialDecay | None = None,
        **kwargs
    ):
        super().__init__(**kwargs)
        self.env = env
        self.buffer = buffer
        self.sampler = sampler
        self.frequency = frequency
        self.steps_per_play = steps_per_play
        self.epsilon_decay = epsilon_decay

        self.observations = None

    def on_epoch_begin(self, epoch: int, logs=None):
        if self.epsilon_decay is not None:
            self.sampler.epsilon = self.epsilon_decay(epoch)
        if epoch == 0:
            model = self.model.get_layer(
                f'{self.model.name}_model'
            )

            self.buffer.clear()
            
            self.observations = play_game(
                model=model,
                buffer=self.buffer,
                env=self.env,
                steps=self.steps_per_play,
                sampler=self.sampler,
                observations=self.observations
            )

    def on_train_batch_begin(self, batch_num: int, logs=None):
        if (batch_num + 1) % self.frequency:
            return
        
        model = self.model.get_layer(
            f'{self.model.name}_model'
        )

        self.observations = play_game(
            model=model,
            buffer=self.buffer,
            env=self.env,
            steps=self.steps_per_play,
            sampler=self.sampler,
            observations=self.observations
        )

# Training

Create models, replay buffers, sampler, optimizer, epsilon decay etc. Implement training loop, show training progress and perform model evaluation once in a while

In [30]:
model, target_model, combined_model = get_combined_model(
    observation_space=train_env.single_observation_space,
    action_space=train_env.single_action_space,
    features=16,
    blocks=4,
    activation='relu',
    dropout_rate=0.2,
    multiply_freq=1,
    name='boxing',
    gamma=0.99
)

In [31]:
buffer_size = 2 ** 15     # 32_768
steps_per_play = 2 ** 11  # 2048
steps_per_epoch = 2 ** 10 # 1024

end_epsilon = 0.1
epochs = 128
decay_epochs = epochs // 2
update_frequency = 512
play_frequency = 256

In [32]:
sampler = Sampler(epsilon=1, seed=42)

In [33]:
epsilon_decay = keras.optimizers.schedules.PolynomialDecay(1., decay_epochs, end_learning_rate=end_epsilon)

In [34]:
train_buffer = ReplayBuffer(
    max_size=buffer_size,
    observation_space=train_env.single_observation_space,
    action_space=train_env.single_action_space
)

In [35]:
dataset = QDataset(
    steps_per_epoch=steps_per_epoch,
    batch_size=train_env.num_envs,
    buffer=train_buffer,
    action_space=train_env.single_action_space
)

In [36]:
# 'foo{epoch}'.format(epoch=42)

checkpoint_filepath = './history/checkpoint.model-{epoch}.keras'

model_checkpoint_callback = keras.callbacks.ModelCheckpoint(
    filepath=checkpoint_filepath,
    monitor='loss',
    mode='min',
)

In [37]:
callbacks = [
    ModelUpdateCallback(update_frequency),
    ModelEvalCallback(valid_env, 1000),
    SometimesPlayCallback(
        env=train_env,
        buffer=train_buffer,
        sampler=sampler,
        frequency=play_frequency,
        steps_per_play=steps_per_play,
        epsilon_decay=epsilon_decay,
    ),
    model_checkpoint_callback
]

In [38]:
combined_model.compile(
    loss=(lambda y_true, y_pred: keras.ops.mean(y_pred)),
    optimizer=keras.optimizers.Adam(1e-3, clipnorm=5, weight_decay=2e-5),
)

In [39]:
_ = play_game(
    model=model,
    buffer=train_buffer,
    env=train_env,
    steps=steps_per_play,
    sampler=sampler,
)

In [40]:
combined_model.fit(dataset, callbacks=callbacks, epochs=epochs)

Epoch 1/128
1024/1024 ━━━━━━━━━━━━━━━━━━━━ 83s 77ms/step - loss: 0.1745 - score: -31.0000
Epoch 2/128


/home/timohavdk/miniconda3/envs/p11/lib/python3.11/site-packages/keras/src/saving/serialization_lib.py:390: UserWarning: The object being serialized includes a `lambda`. This is unsafe. In order to reload the object, you will have to pass `safe_mode=False` to the loading function. Please avoid using `lambda` in the future, and use named Python functions instead. This is the `lambda` being serialized:     loss=(lambda y_true, y_pred: keras.ops.mean(y_pred)),

  return {key: serialize_keras_object(value) for key, value in obj.items()}


1024/1024 ━━━━━━━━━━━━━━━━━━━━ 79s 77ms/step - loss: 1.6662 - score: -39.0000
Epoch 3/128
1024/1024 ━━━━━━━━━━━━━━━━━━━━ 79s 77ms/step - loss: 0.2442 - score: -20.0000
Epoch 4/128
1024/1024 ━━━━━━━━━━━━━━━━━━━━ 80s 78ms/step - loss: 4.6593 - score: -39.0000
Epoch 5/128
1024/1024 ━━━━━━━━━━━━━━━━━━━━ 82s 80ms/step - loss: 0.2404 - score: -63.0000
Epoch 6/128
1024/1024 ━━━━━━━━━━━━━━━━━━━━ 84s 82ms/step - loss: 3.1869 - score: -20.0000
Epoch 7/128
1024/1024 ━━━━━━━━━━━━━━━━━━━━ 79s 77ms/step - loss: 0.1628 - score: 2.0000
Epoch 8/128
1024/1024 ━━━━━━━━━━━━━━━━━━━━ 80s 78ms/step - loss: 1.6639 - score: -66.0000
Epoch 9/128
1024/1024 ━━━━━━━━━━━━━━━━━━━━ 79s 77ms/step - loss: 0.2726 - score: -39.0000
Epoch 10/128
1024/1024 ━━━━━━━━━━━━━━━━━━━━ 81s 79ms/step - loss: 0.1707 - score: -31.0000
Epoch 11/128
1024/1024 ━━━━━━━━━━━━━━━━━━━━ 84s 82ms/step - loss: 0.3239 - score: -31.0000
Epoch 12/128
1024/1024 ━━━━━━━━━━━━━━━━━━━━ 81s 79ms/step - loss: 3.5961 - score: -31.0000
Epoch 13/128
1024/102

In [58]:
train_buffer.rewards.max()

np.float32(2.0)

# Testing

Test the model on the environment and get a cool video

In [21]:
model = keras.models.load_model('./models/model-v3.keras')

I0000 00:00:1744291135.321130   55616 gpu_device.cc:2019] Created device /job:localhost/replica:0/task:0/device:GPU:0 with 5660 MB memory:  -> device: 0, name: NVIDIA GeForce RTX 3050, pci bus id: 0000:09:00.0, compute capability: 8.6


In [22]:
saving_steps = 3

In [23]:
saving_env = gym.make_vec("ALE/Boxing-v5", render_mode='rgb_array_list', num_envs=1)

In [26]:
def save_gameplay(
    model: tf.keras.Model,
    max_steps: int = 1000,
    env: gym.Env | None = None,
):
    save_buffer = ReplayBuffer(
        max_size=200000,
        observation_space=train_env.single_observation_space,
        action_space=train_env.single_action_space
    )
    
    save_sampler = Sampler(0, greedy=True)
    
    _ = play_game(
        model=model,
        env=env,
        steps=max_steps,
        sampler=save_sampler,
        buffer=save_buffer
    )
    
    render = env.render()

    # gym.utils.save_video.save_video(
    #     frames=render[0],
    #     video_folder="videos",
    #     fps=env.metadata["render_fps"],
    # )

    return render

In [52]:
test_env = gym.make_vec("ALE/Boxing-v5", render_mode='rgb_array', num_envs=1)

In [53]:
observation = test_env.reset()

In [56]:
res = model(observation[0])

In [57]:
res

<tf.Tensor: shape=(1, 18), dtype=float32, numpy=
array([[158.84975, 158.6238 , 155.97168, 159.64922, 157.33707, 164.28032,
        156.86633, 156.56297, 158.59987, 157.19196, 158.47374, 157.69778,
        160.59332, 156.52937, 159.25946, 158.40334, 159.13138, 158.55917]],
      dtype=float32)>

In [27]:
res = save_gameplay(model, max_steps=saving_steps, env=saving_env)

I0000 00:00:1744291193.204067   55616 cuda_dnn.cc:529] Loaded cuDNN version 90300


In [33]:
res[0][0].shape

(210, 160, 3)

In [45]:
model.save('./models/model-v3.keras')
target_model.save('./models/target_model-v3.keras')

In [64]:
model_test = get_model(
    observation_space=train_env.single_observation_space,
    action_space=train_env.single_action_space,
    features=32,
    blocks=4,
    activation='relu',
    dropout_rate=0.2,
    multiply_freq=1,
    name='enduro',
)

In [65]:
model_test.summary()

Model: "enduro"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ enduro_Input (InputLayer)       │ (None, 210, 160, 3)    │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ enduro_Rescaling (Rescaling)    │ (None, 210, 160, 3)    │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ enduro_Conv2D_0 (Conv2D)        │ (None, 210, 160, 32)   │           896 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ enduro_MaxPooling2D_0           │ (None, 105, 80, 32)    │             0 │
│ (MaxPooling2D)                  │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ enduro_Conv2D_1 (Conv2D)        │ (None, 105, 80, 64)    │        18,496 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ enduro_MaxPooling2D_1           │ (None, 52, 40, 64)     │             0 │
│ (MaxPooling2D)                  │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ enduro_Conv2D_2 (Conv2D)        │ (None, 52, 40, 192)    │       110,784 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ enduro_MaxPooling2D_2           │ (None, 26, 20, 192)    │             0 │
│ (MaxPooling2D)                  │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ enduro_Conv2D_3 (Conv2D)        │ (None, 26, 20, 768)    │     1,327,872 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ enduro_MaxPooling2D_3           │ (None, 13, 10, 768)    │             0 │
│ (MaxPooling2D)                  │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ enduro_BatchNormalization_3     │ (None, 13, 10, 768)    │         3,072 │
│ (BatchNormalization)            │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ enduro_GlobalAveragePooling2D   │ (None, 768)            │             0 │
│ (GlobalAveragePooling2D)        │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ enduro_Output (Dense)           │ (None, 18)             │        13,842 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 1,474,962 (5.63 MB)

 Trainable params: 1,473,426 (5.62 MB)

 Non-trainable params: 1,536 (6.00 KB)